# Multi-Omics Imputation

The plan is to do the following:
- Divide the data into train, validation, and test sets. Leave the test set for later.
- Remove the values for a certain omics type from the validation set
- Impute using the train set and the remaining value in the validation set
- Compare these imputed values against the true values
    - Distribution of correlation coefficients
    - Get the mean and stdev of the correlation coefficients
    - Choose best model
- Evaluate on test set
- Choose best method
- Try on independent set
- Finally, train GCN model and see difference between single omics, multi-omics, and imputed multi-omics

I'll first start with some basics: data import and processing.
Then I'll move to imputing one omics from two.
Then I'll move to imputing two omics from one.
I'll do all the steps above along the way.

## Importing requisite packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from metrics import *

## Importing data

In [2]:
mrna = pd.read_csv("../R/TCGA BRCA/mrna_top1000.csv", index_col=0)
meth = pd.read_csv("../R/TCGA BRCA/meth_top1000.csv", index_col=0)
mirna = pd.read_csv("../R/TCGA BRCA/mirna_anova.csv", index_col=0)

labels = pd.read_csv("../R/TCGA BRCA/PAM50_subtype.csv", index_col=0)

## Basic Data Processing

Just combining all data and then also having a list containing what datatype the columns belong to.

In [3]:
all_data = pd.merge(pd.merge(mrna, meth, left_index=True, right_index=True), mirna,  left_index=True, right_index=True)

datatypes = ["mrna"]*mrna.shape[1] + ["meth"]*meth.shape[1] + ["mirna"]*mirna.shape[1]

In [4]:
all_data = (all_data-all_data.min())/(all_data.max() - all_data.min())
all_data.head()

,DBF4|10926,DACH1|1602,BBS4|585,L3MBTL4|91133,TK1|7083,KIAA1370|56204,GPD1L|23171,RERG|85004,RAPGEF3|10411,FBXO36|130888,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1XU-01,0.277770,0.758159,0.378836,0.356571,0.434890,0.554010,0.492860,0.812734,0.599057,0.787462,...,0.004411,0.078464,0.3125,0.002911,0.001617,0.059937,0.046753,0.256461,0.010449,0.070690
TCGA-D8-A1XV-01,0.513189,0.830736,0.648579,0.674419,0.566643,0.889481,0.607707,0.797190,0.454187,0.844195,...,0.003170,0.012424,0.1250,0.000333,0.002426,0.082900,0.038961,0.037773,0.003380,0.032759
TCGA-E9-A1N3-01,0.509665,1.000000,0.553200,0.152295,0.632370,0.669888,0.497003,0.789170,0.553367,0.915498,...,0.003446,0.044661,0.6875,0.000832,0.002426,0.063510,0.049351,0.093439,0.053473,0.110345
TCGA-C8-A1HE-01,0.362294,0.879097,0.601582,0.283353,0.540255,0.786171,0.813419,0.758284,0.668789,0.885250,...,0.001792,0.057869,0.1250,0.001248,0.004448,0.066019,0.007792,0.041750,0.001537,0.044828
TCGA-A1-A0SQ-01,0.429836,0.693130,0.505465,0.292312,0.608069,0.628752,0.616969,0.737191,0.509752,0.708378,...,0.001103,0.005261,0.0625,0.000083,0.003235,0.010478,0.005195,0.009940,0.000615,0.155172


In [5]:
labels.head()

,cancer_subtype
patient_id,
TCGA-D8-A1XU-01,LumA
TCGA-D8-A1XV-01,LumA
TCGA-E9-A1N3-01,LumA
TCGA-C8-A1HE-01,LumA
TCGA-A1-A0SQ-01,LumA


Doing the train-validation-test split.
These contain all the values intact.  

Here, we do a 60-20-20 split.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(all_data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42, stratify = y_train)


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

#all_data.head()
#labels.head()
#X_train.head()
#y_train.head()

(372, 2257)
(125, 2257)
(125, 2257)
(372, 1)
(125, 1)
(125, 1)


Removing all miRNA feature values

In [7]:
#Keeping values for later
from copy import deepcopy
X_test_truth = deepcopy(X_test)
X_val_truth = deepcopy(X_val)

mask = [x=="mirna" for x in datatypes]
X_test.loc[:,mask] = np.nan
X_val.loc[:,mask] = np.nan

X_test.loc[:,mask].head()

/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A3XX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0DI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-A7-A6VX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0AZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-AR-A1AX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Simple Imputation Methods

In [12]:
from sklearn.impute import SimpleImputer

#Combining the train and test samples into one dataframe.
print(X_train.shape)
print(X_val.shape)
X = pd.concat([X_train, X_val])
print(X.shape)
X.iloc[370:375, mask]

(372, 2257)
(125, 2257)
(497, 2257)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1JB-01,0.079457,0.046150,0.008850,0.152926,0.099120,0.103930,0.080421,0.056304,0.022416,0.021445,...,0.000965,0.101410,0.0000,0.007236,0.002022,0.022180,0.033766,0.055666,0.006146,0.005172
TCGA-D8-A1JL-01,0.110465,0.138854,0.017699,0.097901,0.139139,0.027866,0.097684,0.076584,0.062983,0.020316,...,0.115024,0.202149,0.5625,0.000250,0.020218,0.083382,0.033766,0.236581,0.003688,0.008621
TCGA-EW-A2FS-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A8FZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D8-A1XG-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Imputing with Mean and Median

### Imputing with Mean

In [13]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)

mean_imputed = imp.transform(X_val)
# SimpleImputers returns a numpy.ndarray
# I will convert it to a pandas data frame
#mean_imputed = pd.DataFrame(mean_imputed, columns = X_val.columns, index = X_val.index)


#print(mean_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#mean_imputed.loc[:,mask].head()

### Imputing with Median

In [14]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X_train)

median_imputed = imp.transform(X_val)
#median_imputed = pd.DataFrame(median_imputed, columns = X_val.columns, index = X_val.index)

#print(median_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#median_imputed.loc[:,mask].head()

In [15]:
mask = [x=="mirna" for x in datatypes]
truth = X_val_truth.loc[:,mask].to_numpy()
random = (np.random.rand(truth.shape[0],truth.shape[1]))# - np.mean(truth))/np.std(truth)

print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, random))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, median_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(random.std())
print(mean_imputed[:,mask].std())
print(median_imputed[:,mask].std())

RMSE
0.0
0.5262687084890254
0.09808654721684143
0.10037655901358895

Standard Deviation
0.10615528918894765
0.2899613916518392
0.04489987012416516
0.03600215901869758


# Slightly More Complicated Methods

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Here, initially, all the missing features are replaced with the mean value. Then, iteratively, the estimator is used to estimate the missing value from all the other features. Within each iteration through all the missing values, the features are imputed in a random order.

## Estimator: ElasticNet

#### L1 Ratio = 0.2

In [13]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.2), initial_strategy = "mean", 
                       imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

elasticnet_l2_imputed = imp.transform(X_val)
#elasticnet_l2_imputed = pd.DataFrame(elasticnet_l2_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l2_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l2_imputed.loc[:,mask].head()

#### L1 Ratio = 0.8

In [14]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.75), initial_strategy = "mean", 
                       imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

elasticnet_l1_imputed = imp.transform(X_val)
#elasticnet_l1_imputed = pd.DataFrame(elasticnet_l1_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l1_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l1_imputed.loc[:,mask].head()

In [15]:
print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, elasticnet_l1_imputed[:,mask]))
print(rmse(truth, elasticnet_l2_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(mean_imputed[:,mask].std())
print(elasticnet_l1_imputed[:,mask].std())
print(elasticnet_l2_imputed[:,mask].std())

RMSE
0.0
0.09808654721684143
0.09808654721684143
0.09808654721684143

Standard Deviation
0.10615528918894765
0.04489987012416516
0.04489987012416516
0.04489987012416516


## Estimator: KNeighborsRegressor

#### K = 25

In [42]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=25), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 25)
imp.fit(X_train)

knn25_iter_imputed = imp.transform(X_val)

#### K = 50

In [43]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=50), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

knn50_iter_imputed = imp.transform(X_val)

#### K = 75

In [44]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=75), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 75)
imp.fit(X_train)

knn75_iter_imputed = imp.transform(X_val)

#### K = 100

In [45]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=100), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 100)
imp.fit(X_train)

knn100_iter_imputed = imp.transform(X_val)

In [46]:
print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, knn25_iter_imputed[:,mask]))
print(rmse(truth, knn50_iter_imputed[:,mask]))
print(rmse(truth, knn75_iter_imputed[:,mask]))
print(rmse(truth, knn100_iter_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(mean_imputed[:,mask].std())
print(knn25_iter_imputed[:,mask].std())
print(knn50_iter_imputed[:,mask].std())
print(knn75_iter_imputed[:,mask].std())
print(knn100_iter_imputed[:,mask].std())

RMSE
0.0
0.09808654721684143
0.09706443121270594
0.09684563410179363
0.09688083879202193
0.09683528846469257

Standard Deviation
0.10615528918894765
0.04489987012416516
0.04442395037412617
0.04136561038910045
0.04102801872375675
0.040607221785942914


## Estimator: RandomForestRegressor

#### Max Depth = 10

In [11]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=10), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 200)
imp.fit(X_train)

rf10_imputed = imp.transform(X_val)

#### Max Depth = 20

In [12]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=20), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 200)
imp.fit(X_train)

rf20_imputed = imp.transform(X_val)

In [13]:
print(nrmse(truth, mean_imputed[:,mask]))
print(nrmse(truth, rf10_imputed[:,mask]))
print(nrmse(truth, rf20_imputed[:,mask]))

1.3514235544024777
1.3526947398648599
1.353721124349755


# Deck Imputation

It is not exactly deck imputation in that we are not replacing the missing value with a value from the existing set. Here, I select the k closest samples and get the average of their values to impute the missing values.

I am testing kNN with k values equal to all odd numbers between 0 and 20.

In [10]:
from sklearn.impute import KNNImputer

knn = {}

for i in [1,5,10,15,20,25,30,35,40,45,50,75,100]:
    imputer = KNNImputer(n_neighbors=i)
    imputer.fit(X_train)

    knn[i] = imputer.transform(X_val)
    #knn[i] = pd.DataFrame(knn[i], columns = X_val.columns, index = X_val.index)

print(len(knn))
#print(knn[1].shape)
#mask = [x=="mirna" for x in datatypes]
#knn[1].loc[:,mask].head()

13


In [16]:
print(rmse(truth, truth))
print(truth.std())
print()
print(rmse(truth, mean_imputed[:,mask]))
print(mean_imputed[:,mask].std())
print()

print(min([rmse(truth, each[:,mask]) for each in knn.values()]))
print(max([each[:,mask].std() for each in knn.values()]))
for each in knn.values():
    print()
    print(rmse(truth, each[:,mask]))
    print(each[:,mask].std())

0.0
0.10615528918894765

0.09808654721684143
0.04489987012416516

0.09772186476154054
0.10478754862122502

0.13112292928554084
0.10478754862122502

0.1047475141839711
0.07203862173704595

0.10097084589363316
0.06427839857282282

0.09977679942888053
0.06140562119523913

0.09925989347980214
0.059305083427441035

0.09856044397313468
0.05781558990162404

0.0980691452730506
0.05642601332976928

0.09803534611140967
0.055640456876138306

0.09816291728792632
0.054956696204829354

0.09784663394111433
0.0542942718341059

0.09772186476154054
0.05382403291660356

0.0977562975188772
0.05158934607844852

0.09773882456164201
0.050021110608876346


# Comparing Imputation Methods

To compare the imputation methods, we first need to quantify them. Here, I am going to use the Normalized Root Mean Squared Error (NRMSE) to quantify each of the methods and then compare them.

< Insert NRMSE formula in latex >

### Truth and Random values' RMSE and Standard Deviation

In [10]:
missing_types = [["mirna"], ["meth"], ["mrna"], ["mirna", "meth"], ["meth", "mrna"], ["mrna", "mirna"]]
df_rows_rmse = []
df_rows_std = []
for missing in missing_types:
    print("\n", "="*50)
    print("Missing datatype = ", missing, "\n")
    
    mask = [x in missing for x in datatypes]
    X_test = deepcopy(X_test_truth)
    X_test.loc[:,mask] = np.nan
    truth = X_test_truth.loc[:,mask].to_numpy()
    random = (np.random.rand(truth.shape[0],truth.shape[1]))
    
    imp = SimpleImputer(missing_values=np.nan, strategy="mean")
    imp.fit(X_train)
    mean = imp.transform(X_test)

    print("RMSE")
    print(rmse(truth, truth))
    print(rmse(truth, random))
    print(rmse(truth, mean[:,mask]))

    print("\nStandard Deviation")
    print(truth.std())
    print(random.std())
    print(mean[:,mask].std())


Missing datatype =  ['mirna'] 

RMSE
0.0
0.5280194095587588
0.08532672527431143

Standard Deviation
0.09404484993522633
0.2880182827063015
0.04489987012416516

Missing datatype =  ['meth'] 

RMSE
0.0
0.38274093860354524
0.13395549781914132

Standard Deviation
0.24898564747309185
0.28949310757803054
0.20719970088331416

Missing datatype =  ['mrna'] 

RMSE
0.0
0.3350945359684059
0.1386394962855829

Standard Deviation
0.17038344803172661
0.28919302838434907
0.09626461413543717

Missing datatype =  ['mirna', 'meth'] 

RMSE
0.0
0.415646271871899
0.1255543078298891

Standard Deviation
0.2919038085324888
0.2885746128844383
0.26100866746264045

Missing datatype =  ['meth', 'mrna'] 

RMSE
0.0
0.35974782374909714
0.13631761684810612

Standard Deviation
0.2135322511269876
0.28870182636280134
0.1619799396547426

Missing datatype =  ['mrna', 'mirna'] 

RMSE
0.0
0.38283436743469623
0.1295363403769125

Standard Deviation
0.23730089687875197
0.28874397002557967
0.19483150610691286


### Spearman Correlation

In [17]:
mask = [x=="mirna" for x in datatypes]
truth = X_test_truth.loc[:,mask]
random = np.random.rand(truth.shape[0],truth.shape[1])
sp_truth = truth.corrwith(truth, method="spearman")
sp_random = truth.corrwith(pd.DataFrame(random, columns = X_test.loc[:,mask].columns, index = X_test.index))

In [18]:
print(truth.shape)
print(random.shape)
sp_random.head()

(125, 257)
(125, 257)


hsa-mir-576      0.119275
hsa-mir-200b     0.178020
hsa-mir-3687    -0.065322
hsa-mir-126     -0.129541
hsa-mir-26a-2    0.014167
dtype: float64

Now, figure out how to combine multiple sp_ pandas series.  
Finally, I need a multi-index list, where the indices are \[datatype missing, method, and feature\]

In [19]:
methods = ["mean", "median", "knn1", "knn50"]#, "enet", "knn_iter", "rf", "knn1", "knn50"]
imputed = {}
imputers = {
    "mean": SimpleImputer(missing_values=np.nan, strategy="mean"),
    "median": SimpleImputer(missing_values=np.nan, strategy="median"),
    #"enet": IterativeImputer(estimator = ElasticNet(l1_ratio=0.5), initial_strategy = "mean", imputation_order = "random", random_state = NUMBER, n_nearest_features = 75),
    #"knn_iter": IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=75), initial_strategy="mean", imputation_order="random", random_state = NUMBER, n_nearest_features = 75),
    #"rf": IterativeImputer(estimator = RandomForestRegressor(max_depth=10), initial_strategy="mean", imputation_order="random", random_state = NUMBER, n_nearest_features = 200),
    "knn1": KNNImputer(n_neighbors=1),
    "knn50": KNNImputer(n_neighbors=50),
    }

missing_types = [["mirna"], ["meth"], ["mrna"]]#, ["mirna", "meth"], ["meth", "mrna"], ["mrna", "mirna"]]
df_list = []
for missing in missing_types:
    print("\n", "="*50)
    print("Missing datatype = ", missing, "\n")
    
    mask = [x in missing for x in datatypes]
    X_test = deepcopy(X_test_truth)
    X_test.loc[:,mask] = np.nan
    truth = X_test_truth.loc[:, mask]
    random = pd.DataFrame(np.random.rand(truth.shape[0], truth.shape[1]), columns = X_test.loc[:,mask].columns, index = X_test.index)

    sp_dict = {"random": random.corrwith(truth)}
    for method in methods:
        #print(method)
        imp = imputers[method]
        imp.fit(X_train)
        imputed_values = imp.transform(X_test)
        imputed_values = pd.DataFrame(imputed_values, columns = X_test.columns, index = X_test.index)
        
        sp_score = imputed_values.loc[:,mask].corrwith(truth)
        sp_dict[method] = sp_score
        #print("\n", "-"*50)
    df = pd.DataFrame(sp_dict)
    df = df.melt(ignore_index=False, var_name="method", value_name=" ".join(missing))
    df.set_index('method', append=True, inplace=True)
    df_list.append(df)

final = pd.concat(df_list, axis = 1)
final = final.melt(ignore_index=False, var_name = "missing_values", value_name = "Spearman")
#final.to_csv("spearman.csv", index_label = ["feature", "method"], na_rep = "0")


Missing datatype =  ['mirna'] 


Missing datatype =  ['meth'] 


Missing datatype =  ['mrna'] 



In [62]:
temp = final.reset_index()
temp[(temp["method"] == "knn50") & (temp["level_0"].isin(mirna.columns) & (temp["missing_values"] == "mirna"))]
sp_score

hsa-mir-576       0.339657
hsa-mir-200b      0.454222
hsa-mir-3687      0.259713
hsa-mir-126       0.189294
hsa-mir-26a-2     0.438230
                    ...   
hsa-mir-16-1      0.240053
hsa-mir-550a-2    0.471678
hsa-mir-24-1      0.248162
hsa-mir-508       0.134291
hsa-mir-642a      0.266245
Length: 257, dtype: float64

In [81]:
from scipy.stats import spearmanr
import pandas as pd

mask = [x=="meth" for x in datatypes]
X_test = deepcopy(X_test_truth)
X_test.loc[:,mask] = np.nan
truth = X_test_truth.loc[:, mask]
random = pd.DataFrame(np.random.rand(truth.shape[0], truth.shape[1]), columns = X_test.loc[:,mask].columns, index = X_test.index)

method = "knn50"
imp = imputers[method]
imp.fit(X_train)
imputed_values = imp.transform(X_test)
imputed_values = pd.DataFrame(imputed_values, columns = X_test.columns, index = X_test.index)

df1 = imputed_values.loc[:,mask]
df2 = truth
coeff = {}
pval = {}
for i in range(df1.shape[1]):           
    corrtest = spearmanr(df1[df1.columns[i]], df2[df2.columns[i]])  
    #print(df1.columns[i]==df2.columns[i])
    coeff[df1.columns[i]] = corrtest[0]
    pval[df1.columns[i]] = corrtest[1]

In [82]:
sum([x<0.05 for x in list(pval.values())])

915

For p-val < 0.05  
- mrna: 924 out of 1000  
- meth: 915 out of 1000  
- mirna: 229 out of 257

That is 91.6% of features having a significant correlation.

There is still the problem of all features' values being stored when only mirna is missing. This leads to too many NaNs and subsequently zeros in the file.  
I'll resolve it in R, before making plots. I'll just add a column with the omics name for the feature in "feature" column. Then I'll remove those rows where the omics name is not inside of the "missing_values" column.